<a href="https://colab.research.google.com/github/kseniashumanskaya/course-work/blob/main/%D0%90%D0%B2%D1%82%D0%BE%D0%BC%D0%B0%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F%20%D0%B3%D0%B5%D0%BD%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D1%8F%20%D1%83%D1%82%D0%BE%D1%87%D0%BD%D1%8F%D1%8E%D1%89%D0%B8%D1%85%20%D0%B2%D0%BE%D0%BF%D1%80%D0%BE%D1%81%D0%BE%D0%B2%20%D0%BE%20%D1%81%D0%BE%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D0%B8%20%D0%B7%D0%B4%D0%BE%D1%80%D0%BE%D0%B2%D1%8C%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install summa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54387 sha256=7e9c6c90c782cce117a09429e8e7e3edfdc163587c9480d76b185d581993a006
  Stored in directory: /root/.cache/pip/wheels/4a/ca/c5/4958614cfba88ed6ceb7cb5a849f9f89f9ac49971616bc919f
Successfully built summa


In [ ]:
!pip install sentence-transformers transformers

In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import T5ForConditionalGeneration, T5Tokenizer, BertTokenizer, BertModel, AutoTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import torch
import spacy
from transformers import BertTokenizer, BertModel
from warnings import filterwarnings as filt

filt('ignore')

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained("bert-base-uncased")
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
nlp = spacy.load("en_core_web_sm")

def get_question(sentence, answer):

  mdl = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_squad_v1')
  tknizer = AutoTokenizer.from_pretrained('ramsrigouthamg/t5_squad_v1')

  text = "context: {} answer: {}".format(sentence,answer)
  max_len = 256
  encoding = tknizer.encode_plus(text,max_length=max_len, pad_to_max_length=False,truncation=True, return_tensors="pt")

  input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

  outs = mdl.generate(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  early_stopping=True,
                                  num_beams=5,
                                  num_return_sequences=1,
                                  no_repeat_ngram_size=2,
                                  max_length=300)


  dec = [tknizer.decode(ids,skip_special_tokens=True) for ids in outs]


  Question = dec[0].replace("question:","")
  Question= Question.strip()
  return Question

def get_embedding(doc):

  bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  bert_model = BertModel.from_pretrained("bert-base-uncased")

  tokens = bert_tokenizer.tokenize(txt)
  token_idx = bert_tokenizer.convert_tokens_to_ids(tokens)
  segment_ids = [1] * len(tokens)

  torch_token = torch.tensor([token_idx])
  torch_segment = torch.tensor([segment_ids])

  return bert_model(torch_token, torch_segment)[-1].detach().numpy()

def get_pos(context):
  doc = nlp(context)
  docs = [d.pos_ for d in doc]
  return docs, context.split()

def get_sent(context):
  doc = nlp(context)
  return list(doc.sents)

def get_vector(doc):
  stop_words = "english"
  n_gram_range = (1, 1)
  cv = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words)
  df = cv.fit_transform([doc])
  feature_names = cv.get_feature_names_out()
  return feature_names


def get_key_words(context, module_type = 't'):
  keywords = []
  top_n = 2
  for txt in get_sent(context):
    keywd = get_vector(str(txt))
    print(f'vectors : {keywd}')
    if module_type == 't':
      doc_embedding = get_embedding(str(txt))
      keywd_embedding = get_embedding(' '.join(keywd))
    else:
      doc_embedding = model.encode([str(txt)])
      keywd_embedding = model.encode(keywd)

    distances = cosine_similarity(doc_embedding, keywd_embedding)
    print(distances)
    keywords += [(keywd[index], str(txt)) for index in distances.argsort()[0][-top_n:]]

  return keywords

txt = """Pneumonia is an infection of the lungs. Many germs (bacteria, fungi, parasites, and viruses) can cause pneumonia.
Common symptoms of pneumonia include: Chest pain when breathing or coughing, Altered mental status (confusion),Cough,Fatigue (feeling tired),
Fever or chills,
Nausea, vomiting, or diarrhea. Some people are at increased risk for pneumonia due to age, other health conditions, and certain behaviors.
Viral infections that commonly cause pneumonia include COVID-19,
Human metapneumovirus (HMPV) infection,
Human parainfluenza virus (HPIV) infection,
Influenza (flu),
Respiratory syncytial virus (RSV) infection,
Rhinovirus infection.
The following viral infections can cause pneumonia, but are a less common cause:
Adenovirus infection,
Chickenpox,
Measles.
"""
get_key_words(txt)

for ans, context in get_key_words(txt, 'st'):
  print('=======================================')
  print()
  print(get_question(context, ans))
  print()

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


What is pneumonia?


What is an infection of the lungs called?


Along with fungi, parasites and viruses, what can cause pneumonia?


What can many germs cause?


Along with vomiting, what is a common symptom of pneumonia?


Chest pain when breathing or coughing is a common symptom of what?


Why are some people at increased risk for pneumonia?


Age, other health conditions, and certain behaviors increase the risk of what?


What is another name for influenza?


COVID-19 is a common viral infection that causes what?


What is a less common viral infection?


What can viral infections cause?



In [ ]:
from summa import keywords
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist
from sentence_transformers import SentenceTransformer
from transformers import T5ForConditionalGeneration, AutoTokenizer, BertTokenizer, BertModel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import torch
import spacy
from warnings import filterwarnings as filt

filt('ignore')

# Загрузка моделей и ресурсов
nltk.download("punkt")
nltk.download("stopwords")
nlp = spacy.load("en_core_web_sm")
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

# Функция для генерации вопросов
def get_question(sentence, answer):
    mdl = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_squad_v1')
    tknizer = AutoTokenizer.from_pretrained('ramsrigouthamg/t5_squad_v1')

    text = "context: {} answer: {}".format(sentence, answer)
    max_len = 256
    encoding = tknizer.encode_plus(text, max_length=max_len, pad_to_max_length=False, truncation=True, return_tensors="pt")

    input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

    outs = mdl.generate(input_ids=input_ids,
                        attention_mask=attention_mask,
                        early_stopping=True,
                        num_beams=5,
                        num_return_sequences=1,
                        no_repeat_ngram_size=2,
                        max_length=300)

    dec = [tknizer.decode(ids, skip_special_tokens=True) for ids in outs]

    Question = dec[0].replace("question:", "")
    Question = Question.strip()
    return Question

# Функция для получения эмбеддинга предложения
def get_embedding(doc):
    bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    bert_model = BertModel.from_pretrained("bert-base-uncased")

    tokens = bert_tokenizer.tokenize(doc)
    token_idx = bert_tokenizer.convert_tokens_to_ids(tokens)
    segment_ids = [1] * len(tokens)

    torch_token = torch.tensor([token_idx])
    torch_segment = torch.tensor([segment_ids])

    return bert_model(torch_token, torch_segment)[-1].detach().numpy()

# Функция для получения частей речи и токенов предложения
def get_pos(context):
    doc = nlp(context)
    docs = [d.pos_ for d in doc]
    return docs, context.split()

# Функция для получения предложений из контекста
def get_sent(context):
    doc = nlp(context)
    return list(doc.sents)

# Функция для получения векторов предложений
def get_vector(doc):
    stop_words = "english"
    n_gram_range = (1, 1)
    cv = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words)
    df = cv.fit_transform([doc])
    feature_names = cv.get_feature_names_out()
    return feature_names

# Функция для получения ключевых слов
def get_key_words(context, module_type='t'):
    keywords = []
    top_n = 5
    for txt in get_sent(context):
        keywd = get_vector(str(txt))
        if module_type == 't':
            doc_embedding = get_embedding(str(txt))
            keywd_embedding = get_embedding(' '.join(keywd))
        else:
            doc_embedding = model.encode([str(txt)])
            keywd_embedding = model.encode(keywd)

        distances = cosine_similarity(doc_embedding, keywd_embedding)
        keywords += [(keywd[index], str(txt)) for index in distances.argsort()[0][-top_n:]]

    return keywords

# Загружаем текст
text = """Pneumonia is an infection of the lungs. Many germs (bacteria, fungi, parasites, and viruses) can cause pneumonia.
Common symptoms of pneumonia include: Chest pain when breathing or coughing, Altered mental status (confusion),Cough,Fatigue (feeling tired),
Fever or chills,
Nausea, vomiting, or diarrhea. Some people are at increased risk for pneumonia due to age, other health conditions, and certain behaviors.
Viral infections that commonly cause pneumonia include COVID-19,
Human metapneumovirus (HMPV) infection,
Human parainfluenza virus (HPIV) infection,
Influenza (flu),
Respiratory syncytial virus (RSV) infection,
Rhinovirus infection.
The following viral infections can cause pneumonia, but are a less common cause:
Adenovirus infection,
Chickenpox,
Measles.
"""

# Генерация ключевых слов
key_words = get_key_words(text)
print("Ключевые слова:")
for word, sentence in key_words:
    print(f"- {word} ({sentence})")

# Генерация предложений
sentences = get_sent(text)
print("\nСгенерированные предложения:")
for sentence in sentences:
    print(f"- {sentence}")

# Генерация вопросов
questions = []
for sentence in sentences:
    question = get_question(sentence.text, "")
    questions.append(question)
print("\nСгенерированные вопросы:")
for question in questions:
    print(f"- {question}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Ключевые слова:
- infection (Pneumonia is an infection of the lungs.)
- bacteria (Many germs (bacteria, fungi, parasites, and viruses) can cause pneumonia.
)
- altered (Common symptoms of pneumonia include: Chest pain when breathing or coughing, Altered mental status (confusion),Cough,Fatigue (feeling tired),
Fever or chills,
Nausea, vomiting, or diarrhea.)
- age (Some people are at increased risk for pneumonia due to age, other health conditions, and certain behaviors.
)
- 19 (Viral infections that commonly cause pneumonia include COVID-19,
Human metapneumovirus (HMPV) infection,
Human parainfluenza virus (HPIV) infection,
Influenza (flu),
Respiratory syncytial virus (RSV) infection,
Rhinovirus infection.
)
- adenovirus (The following viral infections can cause pneumonia, but are a less common cause:
Adenovirus infection,
Chickenpox,
Measles.
)

Сгенерированные предложения:
- Pneumonia is an infection of the lungs.
- Many germs (bacteria, fungi, parasites, and viruses) can cause pneum

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import re

# Load the T5 model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_squad_v1')
tokenizer = T5Tokenizer.from_pretrained('ramsrigouthamg/t5_squad_v1')

def extract_symptoms(text):
    # Define a list of potential symptoms based on the context
    symptoms_keywords = [
        "coughing", "producing mucus"
    ]
    symptoms_found = []

    # Check for occurrences of each symptom keyword in the text
    for keyword in symptoms_keywords:
        if re.search(r'\b' + re.escape(keyword) + r'\b', text.lower()):
            symptoms_found.append(keyword)

    return symptoms_found

def generate_questions(symptoms):
    questions = []
    for symptom in symptoms:
        question = f"Do you have {symptom}?"
        questions.append(question)
    return questions

# Example medical text
medical_text = """
Symptoms of bronchitis include: coughing, producing mucus, wheezing or shortness of breath,
fever, chest pain often described as a burning sensation or tightness.
"""

# Extract symptoms from the text
symptoms = extract_symptoms(medical_text)

# Generate questions for each symptom
questions = generate_questions(symptoms)

# Output the generated questions
for q in questions:
    print(f"Generated question: {q}")

Generated question: Do you have coughing?
Generated question: Do you have producing mucus?


In [ ]:
import random

class QuestionGenerator:
    def __init__(self):
        # Templates for generating questions
        self.templates = {
            "symptom": [
                "How long have you been experiencing {symptom}?",
                "Do you have {symptom}?",
                "What kind of {symptom} are you experiencing?"
            ],
            "disease": [
                "What are the symptoms of {disease}?",
                "Have you ever suffered from {disease}?",
                "How did you treat {disease} in the past?"
            ],
            "treatment": [
                "What treatment have you received for {disease}?",
                "Have you seen any improvement after treating {disease}?",
                "How long have you been undergoing treatment for {disease}?"
            ]
        }

    def generate_question(self, category, **kwargs):
        """Generate a question based on the category and provided data."""
        if category in self.templates:
            template = random.choice(self.templates[category])
            return template.format(**kwargs)
        else:
            return "Category not found."

# Example usage
if __name__ == "__main__":
    generator = QuestionGenerator()

    # Generate questions about various symptoms
    symptom_questions = [
        generator.generate_question("symptom", symptom="fatigue"),
        generator.generate_question("symptom", symptom="nausea"),
        generator.generate_question("symptom", symptom="chest pain"),
        generator.generate_question("symptom", symptom="dizziness"),
        generator.generate_question("symptom", symptom="shortness of breath"),
    ]

    for question in symptom_questions:
        print(question)

    # Generate questions about various diseases
    disease_questions = [
        generator.generate_question("disease", disease="asthma"),
        generator.generate_question("disease", disease="hypertension"),
        generator.generate_question("disease", disease="arthritis"),
        generator.generate_question("disease", disease="migraines"),
        generator.generate_question("disease", disease="allergies"),
    ]

    for question in disease_questions:
        print(question)

    # Generate questions about various treatments
    treatment_questions = [
        generator.generate_question("treatment", disease="diabetes"),
        generator.generate_question("treatment", disease="hypertension"),
        generator.generate_question("treatment", disease="depression"),
        generator.generate_question("treatment", disease="chronic pain"),
        generator.generate_question("treatment", disease="anxiety"),
    ]

    for question in treatment_questions:
        print(question)

What kind of fatigue are you experiencing?
How long have you been experiencing nausea?
Do you have chest pain?
What kind of dizziness are you experiencing?
Do you have shortness of breath?
How did you treat asthma in the past?
Have you ever suffered from hypertension?
How did you treat arthritis in the past?
Have you ever suffered from migraines?
What are the symptoms of allergies?
What treatment have you received for diabetes?
What treatment have you received for hypertension?
Have you seen any improvement after treating depression?
What treatment have you received for chronic pain?
Have you seen any improvement after treating anxiety?


In [ ]:
!pip install transformers torch

In [ ]:
from transformers import T5ForConditionalGeneration, AutoTokenizer
import spacy

# Initialize spaCy model
nlp = spacy.load("en_core_web_sm")

# Function to generate health-related questions
def get_health_question(context):
    mdl = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_squad_v1')
    tknizer = AutoTokenizer.from_pretrained('ramsrigouthamg/t5_squad_v1')

    text = f"Generate a health-related question based on the following context: {context}"
    encoding = tknizer.encode_plus(text, return_tensors="pt", padding=True, truncation=True)

    input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

    # Generate the question
    outs = mdl.generate(input_ids=input_ids,
                        attention_mask=attention_mask,
                        early_stopping=True,
                        num_beams=5,
                        num_return_sequences=3,  # Increase the number of questions generated
                        no_repeat_ngram_size=2,
                        max_length=50)

    questions = [tknizer.decode(output, skip_special_tokens=True) for output in outs]
    return questions

# Updated text focusing on specific symptoms
text = """The patient reports experiencing chest pain, a persistent cough, and fatigue.
He also notices a fever and chills, along with shortness of breath.
It's important to ask how long he has been experiencing these symptoms,
if he has any history of lung diseases, and whether he has seen a doctor recently.
Additionally, we should inquire about any medications he is taking and if he has been in contact with anyone who is sick."""

# Generate sentences from the context
sentences = [sent.text for sent in nlp(text).sents]

# Generate health-related questions
health_questions = []
for sentence in sentences:
    questions = get_health_question(sentence)
    health_questions.extend(questions)

# Remove duplicates and limit to 10 questions
health_questions = list(set(health_questions))[:10]

print("\nGenerated health-related questions:")
for question in health_questions:
    print(f"- {question}")


Generated health-related questions:
- question: What does he notice a fever and chills?
- question: How long has a person been experiencing these symptoms?
- question: What symptoms does the patient report?
- question: What symptoms do the patient report?
- question: What does he notice a shortness of breath?
- question: If a person has been in contact with someone who is sick, who should we ask?
- question: How long has a person been experiencing the symptoms?
- question: What does he notice that causes him shortness of breath?
- question: What does the patient report experiencing?
- question: How long has a person been experiencing symptoms of lung disease?


На основе введенных данный (метод на основе правил)

In [ ]:
import random

# Sample input data about the patient's health
patient_data = {
    "symptoms": ["severe abdominal pain", "nausea", "fever"],
    "previous_conditions": ["asthma", "allergy to penicillin"],
    "medications": ["ibuprofen"],
    "test_results": {"blood_sugar": "high"}
}

# Function to generate questions based on rules and templates
def generate_questions(patient_data):
    questions = set()  # Use a set to avoid duplicates

    # Rule-based questions for symptoms
    if "symptoms" in patient_data:
        for symptom in patient_data["symptoms"]:
            questions.add(f"What type of {symptom} are you experiencing?")
            questions.add(f"Where exactly do you feel {symptom}?")
            questions.add(f"How would you describe the {symptom}?")
            questions.add(f"How long have you been experiencing {symptom}?")

    # Rule-based questions for previous conditions
    if "previous_conditions" in patient_data:
        for condition in patient_data["previous_conditions"]:
            questions.add(f"Have you had any issues related to your {condition} recently?")
            questions.add(f"Are you currently taking any medication for your {condition}?")

    # Rule-based questions for medications
    if "medications" in patient_data:
        for medication in patient_data["medications"]:
            questions.add(f"Have you taken {medication} in the last few days?")

    # Rule-based questions based on test results
    if "test_results" in patient_data and "blood_sugar" in patient_data["test_results"]:
        if patient_data["test_results"]["blood_sugar"] == "high":
            questions.add("What does your diet look like lately?")
            questions.add("Have you been exercising regularly?")

    # Randomly shuffle to vary the order of questions
    questions = list(questions)  # Convert back to list
    random.shuffle(questions)
    return questions

# Generate questions
generated_questions = generate_questions(patient_data)

# Print the generated questions
print("Generated Health-Related Questions:")
for question in generated_questions:
    print(f"- {question}")

Generated Health-Related Questions:
- Where exactly do you feel severe abdominal pain?
- Have you had any issues related to your asthma recently?
- Are you currently taking any medication for your asthma?
- How long have you been experiencing severe abdominal pain?
- How would you describe the severe abdominal pain?
- Have you taken ibuprofen in the last few days?
- Where exactly do you feel fever?
- Where exactly do you feel nausea?
- What does your diet look like lately?
- Have you had any issues related to your allergy to penicillin recently?
- What type of nausea are you experiencing?
- How would you describe the fever?
- How would you describe the nausea?
- How long have you been experiencing fever?
- What type of fever are you experiencing?
- How long have you been experiencing nausea?
- Are you currently taking any medication for your allergy to penicillin?
- Have you been exercising regularly?
- What type of severe abdominal pain are you experiencing?
